### Creación de entidades en Dialogflow CX
Este notebook se encarga de subir, a través de la API cliente para Dialogflow CX, las entidades con los posibles títulos de película, los posibles géneros de película y los nombres de usuario que generamos y ya estarían en el sistema desde el primer momento. La versión de CX no permite subir directamente las entindades como JSON desde la interfaz gráfica y es necesario hacerlo mediante esta API.

Importamos las librerías necesarias

In [5]:
import requests
import pandas as pd
from google.oauth2 import service_account
from google.cloud.dialogflowcx_v3beta1.types import entity_type
from google.auth.transport.requests import AuthorizedSession
import pickle
from ast import literal_eval

Creamos una función que dados los valores de una entidad y también su nombre la crea en dialogflow haciendo uso de la API.

In [3]:
"""Función que crea la entidad en nuestro agente de dialogflow a partir de una lista con los valores y su nombres.
La crea llamando a la API con un mapa que recoge explícitamente todos los posibles valores."""
def createEntityKindMap(values, entity_name):
    entity_json = []
    for value in values:
        entity_value = {}
        entity_value['value'] = value
        entity_value['synonyms'] = [value]
        entity_json.append(entity_value)
    print(entity_json)
    print('***************************************')

    # Credenciales de la cuenta en un fichero ".json" que nos permiten realizar los cambios 
    credentials = service_account.Credentials.from_service_account_file(
        "./recomendador-peliculas-345318-0f1f75bf0b0d.json")
    scoped_credentials = credentials.with_scopes(
        ['https://www.googleapis.com/auth/cloud-platform'])
    authed_session = AuthorizedSession(scoped_credentials)
    kind = entity_type.EntityType.Kind.KIND_MAP

    # Datos de nuestro proyecto en la nube y de nuestroa agente Dialogflow
    project_id = "recomendador-peliculas-345318"
    agent_id = "6632fe66-91f5-445e-b999-677e68270c7a"
    location = "europe-west2"
    response = authed_session.post('https://europe-west2-dialogflow.googleapis.com/v3/projects/'+ project_id + '/locations/' + location + '/agents/' + agent_id + '/entityTypes',
                                    json={
                                            "kind": kind,
                                            "displayName": entity_name,
                                            "entities": entity_json
                                        }
                                    )                        

    response_txt = response.text
    print(response_txt)

### Crear entidad "movie-name"
Extramos los títulos de nuestras películas en el dataset y llamamos a la función anterior para subirlos a la nube con el nombre de entidad "movie-name"

In [3]:
df_movies = pd.read_csv("movies_catalog_clean.csv")
display(df_movies["title"])
titles = df_movies["title"].dropna().unique()
createEntityKindMap(titles, "moviename")

0                           Toy Story
1                             Jumanji
2                    Grumpier Old Men
3                   Waiting to Exhale
4         Father of the Bride Part II
                     ...             
14727         It Stains the Sands Red
14728                  In a Heartbeat
14729                     Cadet Kelly
14730    The Man with the Rubber Head
14731                The One-Man Band
Name: title, Length: 14732, dtype: object

14732
[{'value': 'Toy Story', 'synonyms': ['Toy Story']}, {'value': 'Jumanji', 'synonyms': ['Jumanji']}, {'value': 'Grumpier Old Men', 'synonyms': ['Grumpier Old Men']}, {'value': 'Waiting to Exhale', 'synonyms': ['Waiting to Exhale']}, {'value': 'Father of the Bride Part II', 'synonyms': ['Father of the Bride Part II']}, {'value': 'Heat', 'synonyms': ['Heat']}, {'value': 'Sabrina', 'synonyms': ['Sabrina']}, {'value': 'Tom and Huck', 'synonyms': ['Tom and Huck']}, {'value': 'Sudden Death', 'synonyms': ['Sudden Death']}, {'value': 'GoldenEye', 'synonyms': ['GoldenEye']}, {'value': 'The American President', 'synonyms': ['The American President']}, {'value': 'Dracula: Dead and Loving It', 'synonyms': ['Dracula: Dead and Loving It']}, {'value': 'Balto', 'synonyms': ['Balto']}, {'value': 'Nixon', 'synonyms': ['Nixon']}, {'value': 'Cutthroat Island', 'synonyms': ['Cutthroat Island']}, {'value': 'Casino', 'synonyms': ['Casino']}, {'value': 'Sense and Sensibility', 'synonyms': ['Sense and Sens

### Crear entidad "genre"
Extramos los géneros de nuestro dataset y llamamos a la función anterior para subirlos a la nube con el nombre de entidad "genre"

In [6]:
df = pd.read_csv("movies_metadata.csv", converters={'genres': literal_eval})
genres = set()
for genres_list in df['genres'].tolist():
    for genre_dict in genres_list:
        genres.add(genre_dict["name"])

createEntityKindMap(genres, "genre")

C:\Users\jorvi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[{'value': 'Thriller', 'synonyms': ['Thriller']}, {'value': 'Mardock Scramble Production Committee', 'synonyms': ['Mardock Scramble Production Committee']}, {'value': 'Family', 'synonyms': ['Family']}, {'value': 'Rogue State', 'synonyms': ['Rogue State']}, {'value': 'Mystery', 'synonyms': ['Mystery']}, {'value': 'Crime', 'synonyms': ['Crime']}, {'value': 'History', 'synonyms': ['History']}, {'value': 'Music', 'synonyms': ['Music']}, {'value': 'Animation', 'synonyms': ['Animation']}, {'value': 'Sentai Filmworks', 'synonyms': ['Sentai Filmworks']}, {'value': 'Documentary', 'synonyms': ['Documentary']}, {'value': 'Carousel Productions', 'synonyms': ['Carousel Productions']}, {'value': 'Foreign', 'synonyms': ['Foreign']}, {'value': 'Fantasy', 'synonyms': ['Fantasy']}, {'value': 'Drama', 'synonyms': ['Drama']}, {'value': 'Horror', 'synonyms': ['Horror']}, {'value': 'Romance', 'synonyms': ['Romance']}, {'value': 'Comedy', 'synonyms': ['Comedy']}, {'value': 'Telescene Film Group Productions',

### Crear entidad "username"
Extramos los nombre de usuarios que estan inicialmente en el sistema y llamamos a la función anterior para subirlos a la nube con el nombre de entidad "username"

In [6]:
fileTable = open("nameIDTable.pkl", "rb")
nameIDTable = pickle.load(fileTable)
fileTable.close()
usernames = nameIDTable.keys()
createEntityKindMap(usernames, "username")

[{'value': 'Lashay Robinson', 'synonyms': ['Lashay Robinson']}, {'value': 'Jeremiah Miscione', 'synonyms': ['Jeremiah Miscione']}, {'value': 'Mark Mayo', 'synonyms': ['Mark Mayo']}, {'value': 'Dana Boden', 'synonyms': ['Dana Boden']}, {'value': 'Robert Buskey', 'synonyms': ['Robert Buskey']}, {'value': 'Ellen Rangel', 'synonyms': ['Ellen Rangel']}, {'value': 'Adrian Koch', 'synonyms': ['Adrian Koch']}, {'value': 'John Concepcion', 'synonyms': ['John Concepcion']}, {'value': 'Martin Smith', 'synonyms': ['Martin Smith']}, {'value': 'Joseph Cupples', 'synonyms': ['Joseph Cupples']}, {'value': 'Tommy Poore', 'synonyms': ['Tommy Poore']}, {'value': 'Constance Bell', 'synonyms': ['Constance Bell']}, {'value': 'Harold Pendleton', 'synonyms': ['Harold Pendleton']}, {'value': 'Lourdes Silverwood', 'synonyms': ['Lourdes Silverwood']}, {'value': 'Roger Thompson', 'synonyms': ['Roger Thompson']}, {'value': 'Stephanie Moulton', 'synonyms': ['Stephanie Moulton']}, {'value': 'Carmen Hicks', 'synonyms